In [1]:
import langchain
from langchain.embeddings import CacheBackedEmbeddings,HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.retrievers import BM25Retriever,EnsembleRetriever
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.cache import InMemoryCache
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.callbacks import StdOutCallbackHandler
from langchain import PromptTemplate

/home/work/anaconda3/envs/lc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### 使用LangChain进行数据解析和加载

In [15]:
dir_loader = DirectoryLoader("/home/work/project/MockingBird-main/pdf_data/",
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
docs = dir_loader.load()
print(f"len of documents in :{len(docs)}")

len of documents in :117


#### 使用 RecursiveCharacterTextSplitter 创建文本片段

In [20]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)

esops_documents = text_splitter.transform_documents(docs)
print(f"number of chunks in barbie documents : {len(esops_documents)}")

number of chunks in barbie documents : 330


#### 向量化存储

- 在这里，将利用CacheBackedEmbeddings来防止我们一遍又一遍地重新嵌入类似的查询。
- 结构化文档将被转换为有用的格式，以便在大模型应用中查询、检索和使用
- 这里将使用 FAISS（Facebook AI 相似性搜索）作为向量检索引擎。

In [21]:
store = LocalFileStore("./cache/")
# 提前下载到本地的m3e向量化模型
embed_model_id = '/home/work/var/data/ssr-share-data/m3e-base/'
core_embeddings_model = HuggingFaceEmbeddings(model_name=embed_model_id)
embedder = CacheBackedEmbeddings.from_bytes_store(core_embeddings_model,
                                                  store,
                                                  namespace=embed_model_id)
# Create VectorStore
vectorstore = FAISS.from_documents(esops_documents,embedder)

No sentence-transformers model found with name /home/work/var/data/ssr-share-data/m3e-base/. Creating a new one with MEAN pooling.


#### 创建检索器

In [22]:
bm25_retriever = BM25Retriever.from_documents(esops_documents)
bm25_retriever.k=5

#### 从向量库中检索与用户输入query相似的片段

In [23]:
query = "什么是神经网络？"
embedding_vector = core_embeddings_model.embed_query(query)
print(len(embedding_vector))
#
docs_resp = vectorstore.similarity_search_by_vector(embedding_vector, k=1)
#
for page in docs_resp:
  print(page.page_content)
  print("\n")

768
DeepLearning-Ng
13-4-8 -Ufldl
deeplearning.stanford.edu/wiki/index.php/ #.E4.B8.AD.E6.96.87.E8.AF.91.E8.80.85
wbx5/6
要求解这样的神经网络，需要样本集
 ，其中
 。如果你想预测的输出是多
个的，那这种神经网络很适用。（比如，在医疗诊断应用中，患者的体征指标就可以作为向量的输
入值，而不同的输出值
可以表示不同的疾病存在与否。）
中英文对照
neuralnetworks神经网络
activationfunction激活函数
hyperbolictangent双曲正切函数
biasunits偏置项
activation激活值
forwardpropagation前向传播
feedforwardneuralnetwork前馈神经网络(参照Mitchell的《机器学习》的翻译)
中文译者
孙逊（sunpaofu@foxmail.com），林锋（xlfg@yeah.net），刘鸿鹏飞
（just.dark@foxmail.com）,许利杰（csxulijie@gmail.com）




#### 使用集成检索

In [24]:
faiss_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, faiss_retriever],
                                       weights=[0.5, 0.5])  # 每个检索器的权重

#### 加载大模型

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# device = torch.device("cuda:2")
# 提前下载到本地的大模型
model_name_or_path = "/home/work/var/data/ssr-share-data/Qwen-7B-Chat"

model = AutoModelForCausalLM.from_pretrained(model_name_or_path, device_map="auto", trust_remote_code=True).eval()
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True, trust_remote_code=True)


The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
Try importing flash-attention for faster inference...
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|██████████| 8/8 [00:29<00:00,  3.74s/it]


#### 创建pipeline并初始化

In [25]:
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.1,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)


llm = HuggingFacePipeline(pipeline=pipe)

The model 'QWenLMHeadModel' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'RobertaPreLayerNormFo

#### 设置缓存

In [26]:
langchain.llm_cache = InMemoryCache()

#### 设定输入提示模板

In [27]:
PROMPT_TEMPLATE = '''
你是我的学术导师。你对深度学习领域知识非常擅长，能够给我提供关于深度学习、大模型相关的知识和建议。
根据所提供的信息，尝试回答我提出的问题。
如果你无法根据信息回答问题，请说“没有答案”。
因此，在尝试深入理解上下文并进根据所提供的信息进行回答，不要生成不相关的答案。

上下文: {context}
问题: {question}

答案:
'''
#
input_variables = ['context', 'question']
#
custom_prompt = PromptTemplate(template=PROMPT_TEMPLATE,
                            input_variables=input_variables)

#### 检索chain

In [28]:
handler = StdOutCallbackHandler()

qa_with_sources_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever = vectorstore.as_retriever(search_kwargs={"k": 5}),
    verbose=True,
    callbacks=[handler],
    chain_type_kwargs={"prompt": custom_prompt},
    return_source_documents=True
)


#### 测试

In [29]:
query = "什么是神经网络？"
response = qa_with_sources_chain({"query": query})
print(f"Response generated : \n {response['result']}")
print(f"Source Documents : \n {response['source_documents']}")



> Entering new RetrievalQA chain...

> Finished chain.
Response generated : 
 
































































































































































































































































Source Documents : 
 [Document(page_content='DeepLearning-Ng\n13-4-8 -Ufldl\ndeeplearning.stanford.edu/wiki/index.php/ #.E4.B8.AD.E6.96.87.E8.AF.91.E8.80.85\nwbx5/6\n要求解这样的神经网络，需要样本集\n ，其中\n 。如果你想预测的输出是多\n个的，那这种神经网络很适用。（比如，在医疗诊断应用中，患者的体征指标就可以作为向量的输\n入值，而不同的输出值\n可以表示不同的疾病存在与否。）\n中英文对照\nneuralnetworks神经网络\nactivationfunction激活函数\nhyperbolictangent双曲正切函数\nbiasunits偏置项\nactivation激活值\nforwardpropagation前向传播\nfeedforwardneuralnetwork前馈神经网络(参照Mitchell的《机器学习》的翻译)\n中文译者\n孙逊（sunpaofu@foxmail.com），林锋（xlfg@yeah.net），刘鸿鹏飞\n（just.dark@foxmail.com）,许利杰（csxulijie@gmail.com）', metadata={'source': '/home/work/project/MockingBird-main/pdf_data/deep-learning.pdf', 'page': 8

#### 使用集成检索

In [30]:
handler = StdOutCallbackHandler()

qa_with_sources_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever = ensemble_retriever,
    callbacks=[handler],
    chain_type_kwargs={"prompt": custom_prompt},
    return_source_documents=True
)

In [31]:
query = "什么是神经网络？"
response = qa_with_sources_chain({"query":query})
print(f"Response generated : \n {response['result']}")
print(f"Source Documents : \n {response['source_documents']}")




> Entering new RetrievalQA chain...

> Finished chain.
Response generated : 
 
































































































































































































































































Source Documents : 
 [Document(page_content='DeepLearning-Ng\n13-4-8 -Ufldl\ndeeplearning.stanford.edu/wiki/index.php/ #.E4.B8.AD.E6.96.87.E8.AF.91.E8.80.85\nwbx5/6\n要求解这样的神经网络，需要样本集\n ，其中\n 。如果你想预测的输出是多\n个的，那这种神经网络很适用。（比如，在医疗诊断应用中，患者的体征指标就可以作为向量的输\n入值，而不同的输出值\n可以表示不同的疾病存在与否。）\n中英文对照\nneuralnetworks神经网络\nactivationfunction激活函数\nhyperbolictangent双曲正切函数\nbiasunits偏置项\nactivation激活值\nforwardpropagation前向传播\nfeedforwardneuralnetwork前馈神经网络(参照Mitchell的《机器学习》的翻译)\n中文译者\n孙逊（sunpaofu@foxmail.com），林锋（xlfg@yeah.net），刘鸿鹏飞\n（just.dark@foxmail.com）,许利杰（csxulijie@gmail.com）', metadata={'source': '/home/work/project/MockingBird-main/pdf_data/deep-learning.pdf', 'page': 8